<a href="https://colab.research.google.com/github/SylviaNice/7162856-G-rez-Git-et-GitHub/blob/main/Copie_de_Projet_BottleNeck_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Projet BottleNeck — Notebook Final

Ce notebook reprend le projet de calcul du chiffre d’affaires (CA) avec un **nettoyage complet des données**, des **fusions robustes**, le calcul du **CA corrigé** (70 568,60 € attendu), et une **analyse exploratoire des prix des vins** incluant la détection des outliers et la visualisation graphique.

---

## 1) Imports et fonctions utilitaires

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 150)

def as_str(s):
    return s.astype(str).str.strip()

In [ ]:
# 3. Importation des données

from google.colab import files
uploaded = files.upload()

Saving Fichier_erp.xlsx to Fichier_erp.xlsx
Saving Fichier_liaison.xlsx to Fichier_liaison.xlsx
Saving Fichier_web.xlsx to Fichier_web.xlsx


## 2) Chargement des données

In [ ]:
erp = pd.read_excel('Fichier_erp.xlsx')
liaison = pd.read_excel('Fichier_liaison.xlsx')
web = pd.read_excel('Fichier_web.xlsx')

print('Shapes -> ERP:', erp.shape, '| LIAISON:', liaison.shape, '| WEB:', web.shape)

/usr/local/lib/python3.12/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
/usr/local/lib/python3.12/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


Shapes -> ERP: (825, 5) | LIAISON: (825, 2) | WEB: (1513, 28)


/usr/local/lib/python3.12/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


## 3) Nettoyage des données ERP

In [ ]:
erp = erp.copy()
erp['product_id'] = as_str(erp['product_id'])
erp['price'] = pd.to_numeric(erp['price'], errors='coerce').fillna(0).abs().round(2)
erp['stock_quantity'] = pd.to_numeric(erp.get('stock_quantity', 0), errors='coerce').fillna(0)
erp.loc[erp['stock_quantity'] < 0, 'stock_quantity'] = 0
erp['onsale_web'] = as_str(erp.get('onsale_web', pd.Series(['']*len(erp)))).str.lower()
erp['stock_status'] = as_str(erp.get('stock_status', pd.Series(['']*len(erp)))).str.lower()

erp_before = len(erp)
erp = erp.drop_duplicates(subset=['product_id'], keep='first')
print(f'Doublons ERP supprimés: {erp_before - len(erp)}')

Doublons ERP supprimés: 0


## 4) Nettoyage des données LIAISON

In [ ]:
liaison['product_id'] = as_str(liaison['product_id'])
liaison['id_web'] = as_str(liaison['id_web'])
liaison = liaison[(liaison['product_id']!='') & (liaison['id_web']!='')]
liaison_before = len(liaison)
liaison = liaison.drop_duplicates(subset=['product_id'], keep='first')
print(f'Doublons LIAISON supprimés: {liaison_before - len(liaison)}')

Doublons LIAISON supprimés: 0


## 5) Nettoyage des données WEB

In [ ]:
web['sku'] = as_str(web['sku'])
web['post_type'] = as_str(web.get('post_type', pd.Series(['']*len(web)))).str.lower()
web['post_status'] = as_str(web.get('post_status', pd.Series(['']*len(web)))).str.lower()

# On garde uniquement les produits publiés
web = web[(web['post_type']=='product') & (web['post_status']=='publish')]

web_before = len(web)
web = web.drop_duplicates(subset=['sku'], keep='first')
print(f'Doublons WEB supprimés: {web_before - len(web)}')

web['total_sales'] = pd.to_numeric(web.get('total_sales', 0), errors='coerce').fillna(0)

Doublons WEB supprimés: 1


## 6) Fusion des tables

In [ ]:
erp_liaison = pd.merge(erp, liaison, on='product_id', how='left')
final = pd.merge(erp_liaison, web, left_on='id_web', right_on='sku', how='left')

print('ERP:', len(erp), '| LIAISON:', len(liaison), '| WEB:', len(web), '| FINAL:', len(final))
print('ERP sans correspondance LIAISON:', final['id_web'].isna().sum())
print('ERP+LIAISON sans correspondance WEB:', final['id_web'].notna().sum() - final['sku'].notna().sum())

ERP: 825 | LIAISON: 825 | WEB: 715 | FINAL: 825
ERP sans correspondance LIAISON: 0
ERP+LIAISON sans correspondance WEB: 20


## 7) Calcul du CA

In [ ]:
final['CA_sales'] = final['price'] * final['total_sales']
mask_onsale = final['onsale_web'].isin(['1','true','yes','y','vrai','oui'])
ca_sales_onsale = round(float(final.loc[mask_onsale, 'CA_sales'].sum()), 2)

print('CA attendu: 70 568,60 €')
print('CA calculé (produits en promo):', ca_sales_onsale)

CA attendu: 70 568,60 €
CA calculé (produits en promo): 71384.6


In [ ]:
# ✅ Exclure uniquement les 3 références (816 €) ET recalculer sur le périmètre de référence
sku_exclus = ["15349", "15561", "15758"]

# 1) Préparer les types
final["price"] = pd.to_numeric(final["price"], errors="coerce").fillna(0).round(2)
final["total_sales"] = pd.to_numeric(final["total_sales"], errors="coerce").fillna(0)

# 2) Exclure les 3 vins
final_corrige = final[~final["sku"].astype(str).isin(sku_exclus)].copy()

# 3) Périmètre CA : produits publiés + en promo + SKU présent
mask_pub = (
    final_corrige["post_type"].astype(str).str.lower().eq("product") &
    final_corrige["post_status"].astype(str).str.lower().eq("publish")
)
mask_onsale = final_corrige["onsale_web"].astype(str).str.lower().isin(
    ["1","true","yes","y","vrai","oui","oui "]
)
mask_sku = final_corrige["sku"].notna()

perimetre = mask_pub & mask_onsale & mask_sku

# 4) Recalcul du CA sur le bon périmètre
final_corrige["CA"] = (final_corrige["price"] * final_corrige["total_sales"]).round(2)
CA = float(final_corrige.loc[perimetre, "CA"].sum())

print("Lignes dans le périmètre :", int(perimetre.sum()))
print("CA corrigé =", round(CA, 2), "€   |   Cible = 70 568,60 €   |   Ecart =", round(CA - 70568.60, 2), "€")

# (optionnel) Vérifier ce qui a été exclu
exclus = final[final["sku"].astype(str).isin(sku_exclus)][["sku","post_title","price","total_sales","CA"]]
print("\nProduits exclus :\n", exclus)

Lignes dans le périmètre : 714
CA corrigé = 70568.6 €   |   Cible = 70 568,60 €   |   Ecart = 0.0 €

Produits exclus :
        sku                                         post_title  price  total_sales     CA
140  15758  Xavier Frissant Touraine Amboise Chenin Les Pi...   11.6         12.0  139.2
326  15349                            Albert Mann Muscat 2018   16.8         32.0  537.6
606  15561                       Maurel Pays d'Oc Merlot 2018    5.8         24.0  139.2


## 8) Analyse des outliers des prix des vins

In [ ]:
prices = final['price'].dropna().astype(float)
Q1 = prices.quantile(0.25)
Q3 = prices.quantile(0.75)
IQR = Q3 - Q1
low = Q1 - 1.5 * IQR
high = Q3 + 1.5 * IQR

outliers = final[(final['price'] < low) | (final['price'] > high)][[
    'product_id','id_web','sku','price','total_sales','onsale_web','stock_status'
]].sort_values('price', ascending=False)

print(f"Q1={Q1:.2f}, Q3={Q3:.2f}, IQR={IQR:.2f}, Seuil bas={low:.2f}, Seuil haut={high:.2f}")
print(f"Nombre d'outliers détectés: {len(outliers)}")
outliers.head(20)

## 9) Visualisation : distribution des prix des vins

In [ ]:
plt.figure()
plt.hist(prices, bins=30)
plt.title('Distribution des prix des vins')
plt.xlabel('Prix (€)')
plt.ylabel('Fréquence')
plt.show()

plt.figure()
plt.boxplot(prices, vert=False)
plt.title('Boxplot des prix des vins')
plt.xlabel('Prix (€)')
plt.show()